In [1]:
import warnings; warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
import nltk
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [3]:
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd

In [5]:
path = Path('C:/Users/sappusamy/Documents/SriWK/datasets/IMDB.csv')

In [6]:
dataset = pd.read_csv(path)

In [7]:
dataset.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
dataset = dataset[:250]

In [9]:
dataset.sentiment.value_counts()

negative    137
positive    113
Name: sentiment, dtype: int64

In [10]:
train = dataset[:100]
train.sentiment.value_counts()

negative    58
positive    42
Name: sentiment, dtype: int64

In [11]:
test = dataset[100:].reset_index(drop=True)
test.sentiment.value_counts()

negative    79
positive    71
Name: sentiment, dtype: int64

## NLP: Cleaning & Preprocessing text
- Remove HTML
- Tokenization + Remove punctuation
- Remove stopwords
- Lemmatization or stemming

In [12]:
text = train.review[0]

In [13]:
stop_words = stopwords.words('english')

In [14]:
def remove_html(text):
    return BeautifulSoup(text,'lxml').get_text()

In [15]:
def remove_punctuation(text):
    return "".join([c for c in text if c not in string.punctuation])

In [16]:
def remove_stopwords(text):
    return [w for w in text if w not in stop_words]

In [17]:
def word_lemmatizer(text):
    return [lemmatizer.lemmatize(i) for i in text]

In [18]:
def word_stemmer(text):
    return [stemmer.stem(i) for i in text]

In [19]:
tokenizer = RegexpTokenizer(r'\w+')

In [20]:
lemmatizer = WordNetLemmatizer()

In [21]:
stemmer = PorterStemmer()

In [22]:
t = train['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [23]:
labels = train['sentiment']

set(labels)

{'negative', 'positive'}

In [24]:
labels_index = {'negative':0,'positive':1}

#### Represent each word in ONE-HOT encoding
- as of now, for reducing vector length of input we set min frequency of token to be **SOME INT VALUE**

In [25]:
def token_frequency(l):
    count = {}
    for tokens in l:
        for token in tokens:
            if token not in count:
                count[token]=1
            else:
                count[token]+=1
    return count

In [26]:
tf = token_frequency(t)

In [27]:
sorted(tf.items(),key=lambda x:x[1],reverse=True)[:10]

[('movi', 209),
 ('film', 156),
 ('one', 104),
 ('like', 84),
 ('see', 59),
 ('even', 59),
 ('get', 58),
 ('good', 56),
 ('scene', 55),
 ('go', 54)]

In [28]:
min_freq = 2

In [29]:
def build_vocabulary(tf):
    v = {}
    v['pad']=0
    v['unk']=1
    for token in tf:
        if tf[token]>=min_freq:
            v[token] = len(v)
    return v

In [30]:
vocab = build_vocabulary(tf)

In [31]:
len(vocab)

1665

## Let's convert tokens to index and make vector for each word in a review

In [32]:
tokens = t[0]
tokens[:10],len(tokens)

(['one',
  'review',
  'mention',
  'watch',
  '1',
  'oz',
  'episod',
  'youll',
  'hook',
  'right'],
 168)

In [33]:
indices = [vocab[token] if token in vocab else vocab['unk'] for token in tokens]
len(indices)

168

## Let's convert these indices to one-hot vectors
- shape of input will be sequence_length * vocab_size

In [34]:
x = torch.zeros(len(indices),len(vocab))

In [35]:
x[range(len(indices)),indices]=1

In [36]:
x,x.shape

(tensor([[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]), torch.Size([168, 1665]))

## Equation of RNN:
\begin{equation*}
h_t = tanh( W_{ih}X_t + b_{ih} + W_{hh}h_{t-1} + b_{hh})
\end{equation*}

In [37]:
rnn_cell = nn.RNNCell(len(vocab),100)

In [38]:
for name,p in rnn_cell.named_parameters():
    print(name,p.shape)

weight_ih torch.Size([100, 1665])
weight_hh torch.Size([100, 100])
bias_ih torch.Size([100])
bias_hh torch.Size([100])


In [39]:
h = torch.zeros(1,100)

In [40]:
h

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [41]:
for i in range(len(indices)):
    h = rnn_cell(x[i].view(1,-1),h)

In [42]:
print("**value of hidden vector after iterating over all input time steps**")
h

**value of hidden vector after iterating over all input time steps**


tensor([[ 0.0338,  0.2030, -0.0317,  0.1212, -0.0954,  0.2023, -0.0252,  0.1759,
          0.0280, -0.0647, -0.1002, -0.2748,  0.1378, -0.1062, -0.0938,  0.1268,
          0.0855, -0.0884, -0.0353,  0.1125, -0.1529, -0.0224, -0.0970,  0.0887,
         -0.1953,  0.2198, -0.1126, -0.1194, -0.0228,  0.0083,  0.0334,  0.0451,
          0.2578, -0.0333,  0.0253, -0.1298, -0.1032,  0.0850,  0.0416,  0.2748,
         -0.1233, -0.1222,  0.1226, -0.0876,  0.0214,  0.1448,  0.0715, -0.1146,
         -0.0933,  0.0641, -0.1560,  0.0219,  0.0711,  0.1127,  0.0399,  0.2929,
          0.1157,  0.1337, -0.0430,  0.0921, -0.0098,  0.1768,  0.0964,  0.1148,
          0.1065,  0.1152, -0.0201, -0.1322,  0.0811, -0.0056,  0.0863, -0.1216,
         -0.0200, -0.0467,  0.0941,  0.0971, -0.0695, -0.0328, -0.1126,  0.0755,
         -0.1399,  0.1264,  0.0886, -0.0006,  0.1397, -0.0786,  0.0053, -0.0732,
          0.2332, -0.0433,  0.1511, -0.0179, -0.1454,  0.0597,  0.1681,  0.0844,
          0.1815,  0.1823, -

In RNNCell,
    
    inputs are looped over each time steps

<a href="https://pytorch.org/docs/stable/nn.html?highlight=rnncell#torch.nn.RNNCell">PyTorch RNNCell reference</a>

## RNN for 1 training example

In [43]:
rnn = nn.RNN(len(vocab),100,batch_first=True)

In [44]:
indices = [vocab[token] if token in vocab else vocab['unk'] for token in tokens]

In [45]:
x = torch.zeros(len(indices),len(vocab))

x[range(len(indices)),indices]=1


In [46]:
x = x.expand((1,-1,len(vocab)))
x,x.shape

(tensor([[[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]), torch.Size([1, 168, 1665]))

**Shape of h:-**
    
    h = (num_layers*num_directions, bacth_size, hidden_units)
    
    where num_layers parameter take int value as inputs which build ups STACKED RNN

In [47]:
h = torch.zeros(1,1,100)

In [48]:
output,hidden = rnn(x,h)

In [49]:
output.shape,hidden.shape

(torch.Size([1, 168, 100]), torch.Size([1, 1, 100]))

**output** - output of all time steps of all batches

**hidden** - output of last time step of all batches

In [50]:
classifier = nn.Linear(100,2)

In [51]:
preds = classifier(hidden.view(1,100))
preds

tensor([[-0.1914,  0.1241]], grad_fn=<AddmmBackward>)

## RNN for m training examples

In [52]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [53]:
crit = nn.CrossEntropyLoss()

In [54]:
optimizer = optim.SGD([{'params':rnn.parameters()},{'params':classifier.parameters()}],lr=0.01)

In [55]:
for doc,y in zip(indices,labels):
    x = torch.zeros(len(doc),len(vocab))
    x[range(len(doc)),doc]=1
    x = x.expand((1,-1,len(vocab)))
    
    optimizer.zero_grad()
    
    h = torch.zeros(1,1,100)
    output,hidden = rnn(x,h)
    preds = classifier(hidden.view(1,100))
    
    loss = crit(preds,torch.LongTensor([labels_index[y]]))
    loss.backward()
    
    optimizer.step()
    
    print(loss.item())
    

0.5478092432022095
0.6081416606903076
0.5728379487991333
0.7896133065223694
0.57362961769104
0.5618698596954346
0.5614193081855774
0.949131965637207
0.9032877683639526
0.5481340885162354
0.8621177673339844
0.8069772720336914
0.8193782567977905
0.7865504026412964
0.6081416606903076
0.7942619323730469
0.6361009478569031
0.7926075458526611
0.581184446811676
0.7988814115524292
0.6344434022903442
0.8012036085128784
0.5854486227035522
0.7781751751899719
0.777474045753479
0.6703867316246033
0.6205466985702515
0.7958093881607056
0.7427211403846741
0.6948776841163635
0.6429101824760437
0.6244244575500488
0.8053395748138428
0.6195626258850098
0.7716079354286194
0.7250599265098572
0.7328423261642456
0.7341915369033813
0.6754580736160278
0.7119659781455994
0.6926265954971313
0.6940498352050781
0.6971608400344849
0.6652841567993164
0.7352091073989868
0.7601959109306335
0.6388621926307678
0.6862215399742126
0.7318757772445679
0.690115749835968
0.7886290550231934
0.6736520528793335
0.691016674041748


**Why loop over each document?**

    different documents are of differnt lengths

## Batch-wise processing

**Why do we need padding?**
- sequence length of different documents will be of different length
- so flow will be as follows:<br>
      for doc in documents:
          for token in doc:
              pass to RNNCell
- the above methods has huge complexity
- in order to process in batch-wise, we make all documents length to be of same length

    
    Thus padding comes in, which means pads 0's to documents which has length less than max length

In [56]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [57]:
sequence_lengths = [len(d) for d in indices]

In [58]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

372

In [59]:
X = torch.zeros(len(indices),max_length,len(vocab))

In [60]:
X.shape

torch.Size([100, 372, 1665])

In [61]:
for i in range(len(indices)):
    X[[i],range(len(indices[i])),indices[i]]=1

In [62]:
Y = torch.LongTensor([labels_index[i] for i in labels])

In [63]:
class Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(len(vocab),100,batch_first=True)
        self.classifier = nn.Linear(100,2)
        
    def forward(self,x,h):
        output,hidden = self.rnn(x,h)
        return self.classifier(hidden.view(-1,100))

In [64]:
model = Sentiment()

In [65]:
crit = nn.CrossEntropyLoss()

In [66]:
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [67]:
bs=16

In [68]:
for epoch in range(2):
    for i in range(0,X.shape[0],bs):
        xb = X[i:i+bs]
        yb = Y[i:i+bs]
        h = torch.zeros(1,xb.shape[0],100)
        
        optimizer.zero_grad()
        preds = model(xb,h)
        loss = crit(preds,yb)
        loss.backward()
        optimizer.step()
        print(loss.item())
        

0.6945303082466125
0.6886752843856812
0.7156445980072021
0.6933744549751282
0.6893102526664734
0.6728619337081909
0.6406792998313904
0.7126068472862244
0.7243829369544983
0.6600400805473328
0.7046942114830017
0.6701604723930359
0.6487911343574524
0.614656925201416


## Test

In [69]:
t = test['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [70]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [71]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

450

In [72]:
TY = torch.LongTensor([labels_index[i] for i in test.sentiment])

In [73]:
predictions=[]
for index_num in range(len(test)):

    T = torch.zeros(len(indices[index_num]),len(vocab))
    T[range(len(indices[index_num])),indices[index_num]]=1
    T = T.expand((1,-1,len(vocab)))

    h = torch.zeros(1,T.shape[0],100)
    with torch.no_grad():
        predictions.append(F.softmax(model(T,h)).argmax().item())

In [74]:
predictions = torch.LongTensor(predictions)

In [75]:
predictions

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])

In [76]:
"Accuracy:",(predictions==TY).float().mean()

('Accuracy:', tensor(0.5267))

In [77]:
"Number of positives predicted",(predictions==1).sum()

('Number of positives predicted', tensor(0))

In [78]:
"Number of negatives predicted",(predictions==0).sum()

('Number of negatives predicted', tensor(150))

## Use nn.Embedding

In [98]:
t = train['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [99]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [100]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

372

In [101]:
for i in range(len(indices)):
    if len(indices[i])<max_length:
        indices[i]+=[0]*(max_length-len(indices[i]))

In [102]:
X = torch.LongTensor(indices)
X.shape

torch.Size([100, 372])

In [103]:
Y = torch.LongTensor([labels_index[i] for i in labels])

In [104]:
class Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(len(vocab),200,padding_idx=0)
        self.rnn = nn.RNN(200,100,batch_first=True)
        self.classifier = nn.Linear(100,2)
        
    def forward(self,x,h):
        x = self.emb(x)
        output,hidden = self.rnn(x,h)
        return self.classifier(hidden.view(-1,100))

In [105]:
model = Sentiment()

crit = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr=0.1)

bs=32

for epoch in range(10):
    for i in range(0,X.shape[0],bs):
        xb = X[i:i+bs]
        yb = Y[i:i+bs]
        h = torch.zeros(1,xb.shape[0],100)
        
        optimizer.zero_grad()
        preds = model(xb,h)
        loss = crit(preds,yb)
        loss.backward()
        optimizer.step()
        print(loss.item())
        

0.6932849287986755
0.6955429911613464
0.6845163106918335
0.6550172567367554
0.7173296809196472
0.6731282472610474
0.6598580479621887
0.6215626001358032
0.7384545803070068
0.6664385795593262
0.6526764035224915
0.6095917224884033
0.7493111491203308
0.6631247997283936
0.6503868103027344
0.6052871942520142
0.7536835670471191
0.660831093788147
0.649570643901825
0.6036905646324158
0.7552337050437927
0.6590689420700073
0.6492340564727783
0.6030513048171997
0.7557085752487183
0.6576700210571289
0.6490649580955505
0.6027580499649048
0.7557860016822815
0.6565379500389099
0.6489587426185608
0.6025956273078918
0.7557146549224854
0.6556075215339661
0.6488786935806274
0.6024868488311768
0.7555860877037048
0.6548316478729248
0.6488115787506104
0.6024026870727539


## Test

In [106]:
t = test['review'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [107]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

max_length = len(max(t,key=lambda x:len(x)))
max_length

450

In [108]:
TY = torch.LongTensor([labels_index[i] for i in test.sentiment])

In [109]:
predictions = []
for index_num in range(len(test)):
    T = torch.LongTensor([indices[index_num]])
    h = torch.zeros(1,T.shape[0],100)
    with torch.no_grad():
        predictions.append(F.softmax(model(T,h)).argmax().item())

In [110]:
predictions = torch.LongTensor(predictions)

In [111]:
(predictions==TY).float().mean()

tensor(0.4667)

In [112]:
tp = ((TY==1)*(predictions==1)).sum().item()

tn = ((TY==0)*(predictions==0)).sum().item()

fn = ((TY==1)*(predictions==0)).sum().item()

fp = ((TY==0)*(predictions==1)).sum().item()
tp,tn,fp,tn

(8, 62, 17, 62)

#### Postive label - measures

In [113]:
print("Precision:",tp/(tp+fp))

print("Recall:",tp/(tp+fn))

Precision: 0.32
Recall: 0.11267605633802817


#### Negative label - measures

In [114]:
print("Precision:",tn/(tn+fn))

print("Recall:",tn/(tn+fp))

Precision: 0.496
Recall: 0.7848101265822784


In [115]:
model.rnn.weight_hh_l0.grad

tensor([[-0.0033,  0.0016,  0.0011,  ..., -0.0022, -0.0021, -0.0020],
        [-0.0022,  0.0011,  0.0007,  ..., -0.0015, -0.0014, -0.0014],
        [-0.0035,  0.0017,  0.0012,  ..., -0.0023, -0.0022, -0.0021],
        ...,
        [-0.0015,  0.0007,  0.0005,  ..., -0.0010, -0.0010, -0.0009],
        [-0.0033,  0.0016,  0.0011,  ..., -0.0021, -0.0021, -0.0020],
        [-0.0033,  0.0016,  0.0011,  ..., -0.0021, -0.0021, -0.0020]])

In [116]:
model.rnn.weight_ih_l0.grad

tensor([[-2.4314e-27,  1.1330e-26, -1.0523e-27,  ..., -3.9030e-27,
          4.9431e-27,  7.4282e-27],
        [ 1.6336e-28,  4.5620e-26,  1.2111e-27,  ..., -1.0547e-26,
          2.3158e-26,  2.1793e-26],
        [ 9.2576e-27, -6.4853e-26,  7.7108e-27,  ...,  2.0990e-26,
         -3.0852e-26, -3.7548e-26],
        ...,
        [-3.6814e-27,  2.6545e-26,  2.3781e-27,  ..., -7.2364e-27,
          1.2195e-26,  1.6021e-26],
        [ 1.4002e-27,  1.3441e-26,  3.6188e-27,  ..., -1.6093e-27,
          6.2430e-27,  6.5735e-27],
        [ 3.3216e-27, -1.2877e-26, -1.6718e-27,  ...,  4.1407e-27,
         -6.0222e-27, -8.4487e-27]])